In [1]:
pip install holidays

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.6/184.6 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 81.2 MB/s eta 0:00:00ta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pymeeus: filename=PyMeeus-0.5.11-py3-none-any.whl size=730984 sha256=4ff9552320ad85d188ad9584e3f33f6781dafece17a2bbb06571f35b9e9bb81c
  Stored in directory: /home/ec2-user/.cache/pip/wheels/a0/8b/b2/810ae5a6f970c8be4725353400d643c90de1c0f023a9884ee7
Successfully built pymeeus
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p38/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [79]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import holidays
import time

from Help_Funs import smape, is_holiday

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import tensorflow as tf


s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/train.csv'
file_key_2 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/test.csv'
file_key_3 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/sample_submission.csv'
file_key_4 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/TPSSEP22_GDP_data_2017_to_2021.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

bucket_object_4 = bucket.Object(file_key_4)
file_object_4 = bucket_object_4.get()
file_content_stream_4 = file_object_4.get('Body')

## Reading data-files
train = pd.read_csv(file_content_stream_1)
train['date'] = pd.to_datetime(train['date'], format = '%Y-%m-%d')

test = pd.read_csv(file_content_stream_2)
test['date'] = pd.to_datetime(test['date'], format = '%Y-%m-%d')

submission = pd.read_csv(file_content_stream_3)

for df in [train, test]:
    df['product'] = df['product'].str.replace(' ', '_')
    df['product'] = df['product'].str.replace(':', '_')

product_df = train.groupby(["date","product"])["num_sold"].sum().reset_index()    
    
train_df = train.groupby(['date'])['num_sold'].sum().reset_index()
train_nocovid_df = train_df.loc[~((train_df["date"] >= "2020-03-01") & (train_df["date"] < "2020-06-01"))]
train_df = train_nocovid_df

test_all_df = test.groupby(['date'])['row_id'].first().reset_index().drop(columns = 'row_id')
test_all_df_dates = test_all_df[['date']]



# country_gdp = pd.read_csv(file_content_stream_4)

# important_dates = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 124, 125, 126, 127, 140, 141, 167, 168, 169, 170, 171, 173, 174, 175, 176, 177, 178, 179, 
#                    180, 181, 203, 230, 231, 232, 233, 234, 282, 289, 290, 307, 308, 309, 310, 311, 312, 313, 317, 318, 319, 320, 360, 361, 362, 363, 364, 365]
    
## Basic feature engineering 
train_df = train_df
train_df['weekday'] = train_df['date'].dt.dayofweek
train_df['month'] = train_df['date'].dt.month
train_df['weekend'] = np.where(train_df['weekday'] >= 5, 1, 0)
train_df['dayOfMonth'] = train_df['date'].dt.day
train_df['dayOfYear'] = train_df['date'].dt.dayofyear
train_df['quarter'] = train_df['date'].dt.quarter
# train['year'] = train['date'].dt.year
train_df['month_sin'] = np.sin(train_df['month'] * (2 * np.pi / 12))
# train['month_cos'] = np.cos(train['month'] * (2 * np.pi / 12))
train_df['day_sin'] = np.sin(train_df['dayOfMonth'] * (2 * np.pi / 12))
# train['day_cos'] = np.cos(train['dayOfMonth'] * (2 * np.pi / 12))
# train['important_dates'] = train['dayOfYear'].apply(lambda x: x if x in important_dates else 0)

test_df = test_all_df
test_df['weekday'] = test_df['date'].dt.dayofweek
test_df['month'] = test_df['date'].dt.month
test_df['weekend'] = np.where(test_df['weekday'] >= 5, 1, 0)
test_df['dayOfMonth'] = test_df['date'].dt.day
test_df['dayOfYear'] = test_df['date'].dt.dayofyear
test_df['quarter'] = test_df['date'].dt.quarter
# test['year'] = test['date'].dt.year
test_df['month_sin'] = np.sin(test_df['month'] * (2 * np.pi / 12))
# test['month_cos'] = np.cos(test['month'] * (2 * np.pi / 12))
test_df['day_sin'] = np.sin(test_df['dayOfMonth'] * (2 * np.pi / 12))
# test['day_cos'] = np.cos(test['dayOfMonth'] * (2 * np.pi / 12))
# test['important_dates'] = test['dayOfYear'].apply(lambda x: x if x in important_dates else 0)

# ## Appending GDP
# train = pd.merge(train, country_gdp, on = ['country', 'year'], how = 'left')
# train = train.drop(columns = ['year'], axis = 1)

# test = pd.merge(test, country_gdp, on = ['country', 'year'], how = 'left')
# test = test.drop(columns = ['year'], axis = 1)

# ## Appending holidays
# data_holidays = is_holiday(train, test)
# train = data_holidays[0]
# test = data_holidays[1]

product_df.head()

,date,product,num_sold
0,2017-01-01,Kaggle_Advanced_Techniques,4267
1,2017-01-01,Kaggle_Getting_Started,3492
2,2017-01-01,Kaggle_Recipe_Book,2900
3,2017-01-01,Kaggle_for_Kids__One_Smart_Goose,4693
4,2017-01-02,Kaggle_Advanced_Techniques,3242


In [80]:
def get_holidays(df):
    years_list = [2017, 2018, 2019, 2020, 2021]

    holiday_BE = holidays.CountryHoliday('BE', years = years_list)
    holiday_FR = holidays.CountryHoliday('FR', years = years_list)
    holiday_DE = holidays.CountryHoliday('DE', years = years_list)
    holiday_IT = holidays.CountryHoliday('IT', years = years_list)
    holiday_PL = holidays.CountryHoliday('PL', years = years_list)
    holiday_ES = holidays.CountryHoliday('ES', years = years_list)

    holiday_dict = holiday_BE.copy()
    holiday_dict.update(holiday_FR)
    holiday_dict.update(holiday_DE)
    holiday_dict.update(holiday_IT)
    holiday_dict.update(holiday_PL)
    holiday_dict.update(holiday_ES)

    df['holiday_name'] = df['date'].map(holiday_dict)
    df['is_holiday'] = np.where(df['holiday_name'].notnull(), 1, 0)
    df['holiday_name'] = df['holiday_name'].fillna('Not Holiday')
    
    return df

train_df = get_holidays(train_df)
test_df = get_holidays(test_df)

In [81]:
train_df.head()

,date,num_sold,weekday,month,weekend,dayOfMonth,dayOfYear,quarter,month_sin,day_sin,holiday_name,is_holiday
0,2017-01-01,15352,6,1,1,1,1,1,0.5,0.500000,Nowy Rok,1
1,2017-01-02,11578,0,1,0,2,2,1,0.5,0.866025,Año nuevo (Trasladado),1
2,2017-01-03,10884,1,1,0,3,3,1,0.5,1.000000,Not Holiday,0
3,2017-01-04,10445,2,1,0,4,4,1,0.5,0.866025,Not Holiday,0
4,2017-01-05,9795,3,1,0,5,5,1,0.5,0.500000,Not Holiday,0


In [82]:
test_df.head()

,date,weekday,month,weekend,dayOfMonth,dayOfYear,quarter,month_sin,day_sin,holiday_name,is_holiday
0,2021-01-01,4,1,0,1,1,1,0.5,0.500000,Año nuevo,1
1,2021-01-02,5,1,1,2,2,1,0.5,0.866025,Not Holiday,0
2,2021-01-03,6,1,1,3,3,1,0.5,1.000000,Not Holiday,0
3,2021-01-04,0,1,0,4,4,1,0.5,0.866025,Not Holiday,0
4,2021-01-05,1,1,0,5,5,1,0.5,0.500000,Not Holiday,0


In [84]:
X = train_df.drop(['num_sold', 'date', 'holiday_name'], axis = 1)
Y = train_df['num_sold']
X_test = test_df.drop(['date', 'holiday_name'], axis = 1)

In [41]:
X_test.head()

,weekday,month,weekend,dayOfMonth,dayOfYear,quarter,month_sin,day_sin,is_holiday
0,4,1,0,1,1,1,0.5,0.500000,1
1,5,1,1,2,2,1,0.5,0.866025,0
2,6,1,1,3,3,1,0.5,1.000000,0
3,0,1,0,4,4,1,0.5,0.866025,0
4,1,1,0,5,5,1,0.5,0.500000,0


In [85]:
X.head()

,weekday,month,weekend,dayOfMonth,dayOfYear,quarter,month_sin,day_sin,is_holiday
0,6,1,1,1,1,1,0.5,0.500000,1
1,0,1,0,2,2,1,0.5,0.866025,1
2,1,1,0,3,3,1,0.5,1.000000,0
3,2,1,0,4,4,1,0.5,0.866025,0
4,3,1,0,5,5,1,0.5,0.500000,0


In [86]:
t1 = time.time()
scaler = MinMaxScaler()
kf = KFold(n_splits = 4, shuffle = True, random_state = 888)
score_list_tf = []
test_preds_tf = []
fold = 1

## Defining model 
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(32, input_dim = 9, activation = 'relu'),
        tf.keras.layers.Dense(32, activation = 'relu'),
        tf.keras.layers.Dense(1)
])

model.compile(optimizer = 'adam', loss = 'mean_absolute_error')


for train_index, test_index in kf.split(X, Y):
    
    ## Splitting the data
    X_train , X_val = X.iloc[train_index], X.iloc[test_index]  
    Y_train, Y_val = Y.iloc[train_index], Y.iloc[test_index]    
    
    print("X_train shape is :", X_train.shape, "X_val shape is", X_val.shape)
    
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
    X_val = pd.DataFrame(scaler.fit_transform(X_val), columns = X_val.columns)

    model.fit(X_train, Y_train, verbose = 1, epochs = 30, batch_size = 32, validation_data = (X_val, Y_val))
    result = pd.DataFrame(model.predict(X_val))    
    result.columns = ['pred_num_sold']
    result['pred_num_sold'] = [0 if i <= 0 else i for i in result['pred_num_sold']]
    
    Y_val = pd.DataFrame(Y_val.reset_index(drop = True))
    score = smape(Y_val['num_sold'], result['pred_num_sold'])
    print('Fold ', str(fold), ' result is:', score, '\n')
    score_list_tf.append(score)

    test_preds_tf.append(model.predict(X_test))
    fold += 1

t2 = time.time()
print('TF model with cross validation take : {:.3f} sn.'.format(t2-t1))

X_train shape is : (1026, 9) X_val shape is (343, 9)
Epoch 1/30
33/33 [==============================] - 0s 6ms/step - loss: 9329.2354 - val_loss: 9285.2627
Epoch 2/30
33/33 [==============================] - 0s 2ms/step - loss: 9328.0449 - val_loss: 9283.5527
Epoch 3/30
33/33 [==============================] - 0s 2ms/step - loss: 9325.2891 - val_loss: 9279.1338
Epoch 4/30
33/33 [==============================] - 0s 2ms/step - loss: 9318.5693 - val_loss: 9269.0625
Epoch 5/30
33/33 [==============================] - 0s 2ms/step - loss: 9304.5361 - val_loss: 9249.8838
Epoch 6/30
33/33 [==============================] - 0s 2ms/step - loss: 9280.0781 - val_loss: 9218.5850
Epoch 7/30
33/33 [==============================] - 0s 2ms/step - loss: 9242.0684 - val_loss: 9172.1533
Epoch 8/30
33/33 [==============================] - 0s 2ms/step - loss: 9187.7109 - val_loss: 9107.5596
Epoch 9/30
33/33 [==============================] - 0s 2ms/step - loss: 9113.9473 - val_loss: 9021.3115
Epoch 10/30

In [87]:
mean = sum(score_list_tf) / len(score_list_tf)
variance = sum([((x - mean) ** 2) for x in score_list_tf]) / len(score_list_tf)
res = variance ** 0.5
print("Cross validation mean score:", sum(score_list_tf) / len(score_list_tf))
print("Cross validation score's Standart deviation is:", res)

Cross validation mean score: 13.996551338057358
Cross validation score's Standart deviation is: 2.0811366221518663


In [88]:
product_ratio_df = product_df.pivot(index="date", columns="product", values="num_sold")
product_ratio_df = product_ratio_df.apply(lambda x: x/x.sum(),axis=1)
product_ratio_df = product_ratio_df.stack().rename("ratios").reset_index()

product_ratio_2017 = product_ratio_df.loc[product_ratio_df['date'].dt.year == 2017].copy()
product_ratio_2017['mm-dd'] = product_ratio_2017['date'].dt.strftime('%m-%d')
product_ratio_2017 = product_ratio_2017.drop(columns='date')
product_ratio_2017 = product_ratio_2017.reset_index()

product_ratio_2018 = product_ratio_df.loc[product_ratio_df['date'].dt.year == 2018].copy()
product_ratio_2018['mm-dd'] = product_ratio_2018['date'].dt.strftime('%m-%d')
product_ratio_2018 = product_ratio_2018.drop(columns='date')
product_ratio_2018 = product_ratio_2018.reset_index()

product_ratio_2019 = product_ratio_df.loc[product_ratio_df['date'].dt.year == 2019].copy()
product_ratio_2019['mm-dd'] = product_ratio_2019['date'].dt.strftime('%m-%d')
product_ratio_2019 = product_ratio_2019.drop(columns='date')

product_ratio_2019 = product_ratio_2019.reset_index()

product_ratio_2019['mean_ratios'] = (product_ratio_2017['ratios']+ product_ratio_2018['ratios'] + product_ratio_2019['ratios']) / 3
product_ratio_2019.head()

,index,product,ratios,mm-dd,mean_ratios
0,2920,Kaggle_Advanced_Techniques,0.279273,01-01,0.279244
1,2921,Kaggle_Getting_Started,0.230832,01-01,0.227723
2,2922,Kaggle_Recipe_Book,0.188119,01-01,0.189136
3,2923,Kaggle_for_Kids__One_Smart_Goose,0.301776,01-01,0.303897
4,2924,Kaggle_Advanced_Techniques,0.281654,01-02,0.283298


In [89]:
test_product_ratio_df = test.copy()
test_product_ratio_df['mm-dd'] = test_product_ratio_df['date'].dt.strftime('%m-%d')

test_product_ratio_df = pd.merge(test_product_ratio_df, product_ratio_2019, how = "left", on = ["mm-dd", "product"])
test_product_ratio_df.head()

,row_id,date,country,store,product,mm-dd,index,ratios,mean_ratios
0,70128,2021-01-01,Belgium,KaggleMart,Kaggle_Advanced_Techniques,01-01,2920,0.279273,0.279244
1,70129,2021-01-01,Belgium,KaggleMart,Kaggle_Getting_Started,01-01,2921,0.230832,0.227723
2,70130,2021-01-01,Belgium,KaggleMart,Kaggle_Recipe_Book,01-01,2922,0.188119,0.189136
3,70131,2021-01-01,Belgium,KaggleMart,Kaggle_for_Kids__One_Smart_Goose,01-01,2923,0.301776,0.303897
4,70132,2021-01-01,Belgium,KaggleRama,Kaggle_Advanced_Techniques,01-01,2920,0.279273,0.279244


In [94]:
test_product_ratio_df.shape

(17520, 9)

In [91]:
test_preds_tf = pd.DataFrame(np.concatenate(test_preds_tf, axis = 1))
print(test_preds_tf.shape)

test_preds_tf = test_preds_tf.mean(axis = 1)
print(test_preds_tf.head())

(365, 4)
0    18043.660156
1    23612.062500
2    28471.648438
3    19994.648438
4    24118.671875
dtype: float32


In [104]:
test_preds_tf

0       18043.660156
1       23612.062500
2       28471.648438
3       19994.648438
4       24118.671875
           ...      
360    599239.187500
361    603702.812500
362    607826.937500
363    611754.812500
364    615682.812500
Length: 365, dtype: float32

In [105]:
test

,row_id,date,country,store,product
0,70128,2021-01-01,Belgium,KaggleMart,Kaggle_Advanced_Techniques
1,70129,2021-01-01,Belgium,KaggleMart,Kaggle_Getting_Started
2,70130,2021-01-01,Belgium,KaggleMart,Kaggle_Recipe_Book
3,70131,2021-01-01,Belgium,KaggleMart,Kaggle_for_Kids__One_Smart_Goose
4,70132,2021-01-01,Belgium,KaggleRama,Kaggle_Advanced_Techniques
...,...,...,...,...,...
17515,87643,2021-12-31,Spain,KaggleMart,Kaggle_for_Kids__One_Smart_Goose
17516,87644,2021-12-31,Spain,KaggleRama,Kaggle_Advanced_Techniques
17517,87645,2021-12-31,Spain,KaggleRama,Kaggle_Getting_Started
17518,87646,2021-12-31,Spain,KaggleRama,Kaggle_Recipe_Book


In [107]:
test_df

,date,weekday,month,weekend,dayOfMonth,dayOfYear,quarter,month_sin,day_sin,holiday_name,is_holiday
0,2021-01-01,4,1,0,1,1,1,5.000000e-01,5.000000e-01,Año nuevo,1
1,2021-01-02,5,1,1,2,2,1,5.000000e-01,8.660254e-01,Not Holiday,0
2,2021-01-03,6,1,1,3,3,1,5.000000e-01,1.000000e+00,Not Holiday,0
3,2021-01-04,0,1,0,4,4,1,5.000000e-01,8.660254e-01,Not Holiday,0
4,2021-01-05,1,1,0,5,5,1,5.000000e-01,5.000000e-01,Not Holiday,0
...,...,...,...,...,...,...,...,...,...,...,...
360,2021-12-27,0,12,0,27,361,4,-2.449294e-16,1.000000e+00,Not Holiday,0
361,2021-12-28,1,12,0,28,362,4,-2.449294e-16,8.660254e-01,Not Holiday,0
362,2021-12-29,2,12,0,29,363,4,-2.449294e-16,5.000000e-01,Not Holiday,0
363,2021-12-30,3,12,0,30,364,4,-2.449294e-16,2.388680e-15,Not Holiday,0


In [108]:
test

,row_id,date,country,store,product
0,70128,2021-01-01,Belgium,KaggleMart,Kaggle_Advanced_Techniques
1,70129,2021-01-01,Belgium,KaggleMart,Kaggle_Getting_Started
2,70130,2021-01-01,Belgium,KaggleMart,Kaggle_Recipe_Book
3,70131,2021-01-01,Belgium,KaggleMart,Kaggle_for_Kids__One_Smart_Goose
4,70132,2021-01-01,Belgium,KaggleRama,Kaggle_Advanced_Techniques
...,...,...,...,...,...
17515,87643,2021-12-31,Spain,KaggleMart,Kaggle_for_Kids__One_Smart_Goose
17516,87644,2021-12-31,Spain,KaggleRama,Kaggle_Advanced_Techniques
17517,87645,2021-12-31,Spain,KaggleRama,Kaggle_Getting_Started
17518,87646,2021-12-31,Spain,KaggleRama,Kaggle_Recipe_Book


In [109]:
test_df['num_sold'] = test_preds_tf

In [110]:
test_df

,date,weekday,month,weekend,dayOfMonth,dayOfYear,quarter,month_sin,day_sin,holiday_name,is_holiday,num_sold
0,2021-01-01,4,1,0,1,1,1,5.000000e-01,5.000000e-01,Año nuevo,1,18043.660156
1,2021-01-02,5,1,1,2,2,1,5.000000e-01,8.660254e-01,Not Holiday,0,23612.062500
2,2021-01-03,6,1,1,3,3,1,5.000000e-01,1.000000e+00,Not Holiday,0,28471.648438
3,2021-01-04,0,1,0,4,4,1,5.000000e-01,8.660254e-01,Not Holiday,0,19994.648438
4,2021-01-05,1,1,0,5,5,1,5.000000e-01,5.000000e-01,Not Holiday,0,24118.671875
...,...,...,...,...,...,...,...,...,...,...,...,...
360,2021-12-27,0,12,0,27,361,4,-2.449294e-16,1.000000e+00,Not Holiday,0,599239.187500
361,2021-12-28,1,12,0,28,362,4,-2.449294e-16,8.660254e-01,Not Holiday,0,603702.812500
362,2021-12-29,2,12,0,29,363,4,-2.449294e-16,5.000000e-01,Not Holiday,0,607826.937500
363,2021-12-30,3,12,0,30,364,4,-2.449294e-16,2.388680e-15,Not Holiday,0,611754.812500


In [111]:
test_sub_df = pd.merge(test, test_df, how  = "left")
test_sub_df["ratios"] = test_product_ratio_df["ratios"]
test_sub_df["mean_ratios"] = test_product_ratio_df["mean_ratios"]
test_sub_df.head()
# submission.to_csv('TF_submission_10.csv', index = False)
# submission.head()

,row_id,date,country,store,product,weekday,month,weekend,dayOfMonth,dayOfYear,quarter,month_sin,day_sin,holiday_name,is_holiday,num_sold,ratios,mean_ratios
0,70128,2021-01-01,Belgium,KaggleMart,Kaggle_Advanced_Techniques,4,1,0,1,1,1,0.5,0.5,Año nuevo,1,18043.660156,0.279273,0.279244
1,70129,2021-01-01,Belgium,KaggleMart,Kaggle_Getting_Started,4,1,0,1,1,1,0.5,0.5,Año nuevo,1,18043.660156,0.230832,0.227723
2,70130,2021-01-01,Belgium,KaggleMart,Kaggle_Recipe_Book,4,1,0,1,1,1,0.5,0.5,Año nuevo,1,18043.660156,0.188119,0.189136
3,70131,2021-01-01,Belgium,KaggleMart,Kaggle_for_Kids__One_Smart_Goose,4,1,0,1,1,1,0.5,0.5,Año nuevo,1,18043.660156,0.301776,0.303897
4,70132,2021-01-01,Belgium,KaggleRama,Kaggle_Advanced_Techniques,4,1,0,1,1,1,0.5,0.5,Año nuevo,1,18043.660156,0.279273,0.279244


In [112]:
test_sub_df.tail()

,row_id,date,country,store,product,weekday,month,weekend,dayOfMonth,dayOfYear,quarter,month_sin,day_sin,holiday_name,is_holiday,num_sold,ratios,mean_ratios
17515,87643,2021-12-31,Spain,KaggleMart,Kaggle_for_Kids__One_Smart_Goose,4,12,0,31,365,4,-2.449294e-16,-0.5,Not Holiday,0,615682.8125,0.306473,0.304394
17516,87644,2021-12-31,Spain,KaggleRama,Kaggle_Advanced_Techniques,4,12,0,31,365,4,-2.449294e-16,-0.5,Not Holiday,0,615682.8125,0.277440,0.279852
17517,87645,2021-12-31,Spain,KaggleRama,Kaggle_Getting_Started,4,12,0,31,365,4,-2.449294e-16,-0.5,Not Holiday,0,615682.8125,0.224633,0.224372
17518,87646,2021-12-31,Spain,KaggleRama,Kaggle_Recipe_Book,4,12,0,31,365,4,-2.449294e-16,-0.5,Not Holiday,0,615682.8125,0.191453,0.191381
17519,87647,2021-12-31,Spain,KaggleRama,Kaggle_for_Kids__One_Smart_Goose,4,12,0,31,365,4,-2.449294e-16,-0.5,Not Holiday,0,615682.8125,0.306473,0.304394


In [113]:
store_weights = train.groupby('store')['num_sold'].sum() / train['num_sold'].sum()
store_weights

store
KaggleMart    0.742515
KaggleRama    0.257485
Name: num_sold, dtype: float64

In [114]:
country_weights = train.groupby('country')['num_sold'].sum() / train['num_sold'].sum()
country_weights

country
Belgium    0.204104
France     0.184381
Germany    0.204498
Italy      0.161540
Poland     0.098457
Spain      0.147020
Name: num_sold, dtype: float64

In [115]:
def disaggregate_forecast(df) :
    new_df = df.copy()
    
    store_weights = train.groupby('store')['num_sold'].sum() / train['num_sold'].sum()
    print(store_weights)
    country_weights = pd.Series(index = test_sub_df["country"].unique(),data = 1/6)
    print(country_weights)
    for country in country_weights.index:
        new_df.loc[(new_df["country"] == country), "num_sold"] = new_df.loc[(new_df["country"] == country), "num_sold"] *  country_weights[country]
    print(new_df)
    for store in store_weights.index:
        new_df.loc[new_df["store"] == store, "num_sold"] = new_df.loc[new_df["store"] == store, "num_sold"] * store_weights[store]
        
    #new_df["num_sold"] = new_df["num_sold"] * new_df["ratios"]
    new_df["num_sold"] = new_df["num_sold"] * new_df["mean_ratios"]
    new_df["num_sold"] = new_df["num_sold"].round()
    new_df = new_df.drop(columns=["ratios"])    
    
    return new_df

In [116]:
final_df = disaggregate_forecast(test_sub_df)
final_df

store
KaggleMart    0.742515
KaggleRama    0.257485
Name: num_sold, dtype: float64
Belgium    0.166667
France     0.166667
Germany    0.166667
Italy      0.166667
Poland     0.166667
Spain      0.166667
dtype: float64
       row_id       date  country       store  \
0       70128 2021-01-01  Belgium  KaggleMart   
1       70129 2021-01-01  Belgium  KaggleMart   
2       70130 2021-01-01  Belgium  KaggleMart   
3       70131 2021-01-01  Belgium  KaggleMart   
4       70132 2021-01-01  Belgium  KaggleRama   
...       ...        ...      ...         ...   
17515   87643 2021-12-31    Spain  KaggleMart   
17516   87644 2021-12-31    Spain  KaggleRama   
17517   87645 2021-12-31    Spain  KaggleRama   
17518   87646 2021-12-31    Spain  KaggleRama   
17519   87647 2021-12-31    Spain  KaggleRama   

                                product  weekday  month  weekend  dayOfMonth  \
0            Kaggle_Advanced_Techniques        4      1        0           1   
1                Kaggle_Getting_S

,row_id,date,country,store,product,weekday,month,weekend,dayOfMonth,dayOfYear,quarter,month_sin,day_sin,holiday_name,is_holiday,num_sold,mean_ratios
0,70128,2021-01-01,Belgium,KaggleMart,Kaggle_Advanced_Techniques,4,1,0,1,1,1,5.000000e-01,0.5,Año nuevo,1,624.0,0.279244
1,70129,2021-01-01,Belgium,KaggleMart,Kaggle_Getting_Started,4,1,0,1,1,1,5.000000e-01,0.5,Año nuevo,1,508.0,0.227723
2,70130,2021-01-01,Belgium,KaggleMart,Kaggle_Recipe_Book,4,1,0,1,1,1,5.000000e-01,0.5,Año nuevo,1,422.0,0.189136
3,70131,2021-01-01,Belgium,KaggleMart,Kaggle_for_Kids__One_Smart_Goose,4,1,0,1,1,1,5.000000e-01,0.5,Año nuevo,1,679.0,0.303897
4,70132,2021-01-01,Belgium,KaggleRama,Kaggle_Advanced_Techniques,4,1,0,1,1,1,5.000000e-01,0.5,Año nuevo,1,216.0,0.279244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,87643,2021-12-31,Spain,KaggleMart,Kaggle_for_Kids__One_Smart_Goose,4,12,0,31,365,4,-2.449294e-16,-0.5,Not Holiday,0,23192.0,0.304394
17516,87644,2021-12-31,Spain,KaggleRama,Kaggle_Advanced_Techniques,4,12,0,31,365,4,-2.449294e-16,-0.5,Not Holiday,0,7394.0,0.279852
17517,87645,2021-12-31,Spain,KaggleRama,Kaggle_Getting_Started,4,12,0,31,365,4,-2.449294e-16,-0.5,Not Holiday,0,5928.0,0.224372
17518,87646,2021-12-31,Spain,KaggleRama,Kaggle_Recipe_Book,4,12,0,31,365,4,-2.449294e-16,-0.5,Not Holiday,0,5057.0,0.191381


In [119]:
submission['num_sold'] = final_df['num_sold'].astype(int)
submission.to_csv('TF_submission_12.csv', index = False)
submission.head()

,row_id,num_sold
0,70128,624
1,70129,508
2,70130,422
3,70131,679
4,70132,216
